In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
import Utils
from Utils import Constants
import cv2
from facenet_pytorch import InceptionResnetV1
from Models import *
from DataLoaders import *

In [3]:
def get_all_labels():
    labels = ['train_data_clean.csv','validation_data_clean.csv']
    dfs = [pd.read_csv(f) for f in labels]
    df = pd.concat(dfs,axis=0).reset_index().drop('index',axis=1)
    return df
all_labels = get_all_labels()
print(all_labels.shape[0]/100)
all_labels

85.53


,name,skin_tone,gender,age,is_face
0,TRAIN0001.png,0,0,1,False
1,TRAIN0002.png,5,1,0,True
2,TRAIN0005.png,1,1,0,False
3,TRAIN0007.png,1,0,1,True
4,TRAIN0009.png,7,0,1,False
...,...,...,...,...,...
8548,TRAIN11332.png,3,0,1,False
8549,TRAIN2383.png,2,0,2,False
8550,TRAIN6490.png,1,1,0,True
8551,TRAIN9278.png,4,1,1,True


In [4]:
class UnsupervisedTripletEncoder(BaseModel):
    #model to use as a triplet loss
    #will tak in list of three image batchs
    #returns list of tree embeedidng batchs + predictions on first batch of images
    def __init__(self,
                 base_model = None,
                 feature_extractor = None,
                 hidden_dims = [400],
                 embedding_dropout=.3,
                 base_name='model',
                 fine_tune=False,
                 **kwargs):
        super(UnsupervisedTripletEncoder,self).__init__()
                               
        if base_model is None:
            base_model = InceptionResnetV1(pretrained='vggface2')
            base_name = 'dualfacenet'
        else:
            base_name = base_model.get_identifier()
        
        
        if feature_extractor is None:
            feature_extractor = InceptionResnetV1(pretrained='vggface2')
        for param in feature_extractor.parameters():
            param.requires_grad = fine_tune
        for param in base_model.parameters():
            param.requires_grad = True
            
        self.base_model = base_model
        self.feature_extractor = feature_extractor
        
        self.embedding_dropout = torch.nn.Dropout(p=embedding_dropout)
        curr_dim = base_model.logits.in_features + feature_extractor.logits.in_features
        hidden_layers = []
        
        for i,size in enumerate(hidden_dims):
            layer = torch.nn.Linear(curr_dim, size)
            curr_dim = size
            hidden_layers.append(layer)
            hidden_layers.append(torch.nn.ReLU())
            
        self.hidden_layers = torch.nn.ModuleList(hidden_layers)
        
        self.embedding_size = hidden_dims[-1]
        self.norm = torch.nn.BatchNorm1d(self.embedding_size)
        
        def add_dims(n,dims,prefix):
            for dim in dims:
                n += '_'+prefix+str(dim)
            return n
        
        name_string = 'unsupervised_encoder_' + base_name
        name_string = add_dims(name_string,hidden_dims,'h')
        name_string += '_ed' + str(embedding_dropout).replace('0.','')
        
        
    def forward(self,x):
        xb = self.base_model(x)
        xf = self.feature_extractor(x)
        x = torch.cat((xb,xf),axis=-1)
        x = self.embedding_dropout(x)
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.norm(x)
        return x

In [5]:
import scipy.spatial

def get_embeddings(model,data_loader,device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    format_y = lambda y: y.long().to(device)
    model.train(False)
    embeddings = []
    labels = [[],[],[]]
    for i, [x_batch,y_batch] in enumerate(data_loader):
        x_batch = x_batch.to(device)
        embedding = model(x_batch)
        embeddings.append(embedding.detach())
        for ii,yy in enumerate(y_batch):
            labels[ii].append(format_y(yy).detach())
        print(i,end='\r')
    embeddings = torch.cat(embeddings).to(device)
    labels = [torch.cat(l).to(device) for l in labels]
    return embeddings, labels

def cuda_mode(vector):
    unique, counts = vector.unique(return_counts=True)
    return unique[torch.argmax(counts)]

def torch_knn(x, y, k=10):
    #n x n distance matrix
    dists = torch.cdist(x,x)
    #indices for neighbors, self will always be first so skip it
    neighbors = torch.argsort(dists,dim=1,descending=False)[:,1:k+1]
    predictions = torch.zeros(y.shape).long()
    y = y.long()
    for i,n in enumerate(neighbors):
        ny = y[n]
        predictions[i] = cuda_mode(ny)
    accuracy = torch.mean((y == predictions.to(y.get_device())).float()).item()
    return predictions, accuracy

def eval_knn_model(model,val_dataloader,klist=[5],device=None):
    with torch.no_grad():
        val_x, val_y = get_embeddings(model,val_dataloader,device=device)
        results = []
        for k in klist:
            outputs = [torch_knn(val_x, vy,k=k) for vy in val_y]
            accuracys = [i[1] for i in outputs]
            results.append(accuracys)
        if len(klist) < 2:
            results = results[0]
    return results

# testmodel = UnsupervisedTripletEncoder()
# testdata = UnsupervisedTripletGenerator(all_labels,Constants.data_root,upsample=False,batch_size=100,validation=True)
# eval_knn_model(testmodel,testdata)

In [ ]:
def save_train_history(model,history,root=''):
    model_name = model.get_identifier()
    
    df = pd.DataFrame(history)
    df['model'] = model_name
    string = root + 'results/history_' + model_name + '.csv'
    df.to_csv(string,index=False)
    print('saved history to',string)
    return df, string

def train_model(model,
                df,
                root,
                epochs=300,
                lr=.001,
                batch_size=200,
                patience = 20,
                save_path=None,
                histogram =False,
                upsample=True,
                k_values=[3,5,7],
                **kwargs,
               ):
    if save_path is None:
        save_path = root + 'models/'+ model.get_identifier()
        if upsample:
            save_path += '_balanced'
    if upsample:
        patience = int(patience/5) + 1
    data_loader = UnsupervisedTripletGenerator(df,Constants.data_root,batch_size=batch_size,upsample=False,**kwargs)
    validation_loader = UnsupervisedTripletGenerator(df,Constants.data_root,batch_size=batch_size,upsample=False,validation=True,**kwargs)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train(True)
    
    triplet_loss = torch.nn.TripletMarginLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    format_y = lambda y: y.long()#.to(device)
    
    def format_batch(inputs,grad=True):
        xb = []
        for xin in inputs:
            xin = xin.to(device)
            xin.requires_grad_(grad)
            xb.append(xin)
        return xb
    
    def embedding_step(m,xbatch): 
        base = m(xbatch[0])
        positive = m(xbatch[1])
        negative = m(xbatch[2])
        loss = triplet_loss(base,positive,negative)
        return base,loss
    
    def train_epoch():
        running_loss = 0
        running_accuracy = [0,0,0]
        curr_loss = 0
        count = 0
        for i, [x_batch,y_batch] in enumerate(data_loader):
            x_batch = format_batch(x_batch)
            optimizer.zero_grad()
            embedding,embedding_loss = embedding_step(model, x_batch)
            embedding_loss.backward()
            optimizer.step()
            running_loss += embedding_loss.item()
            print('curr loss', embedding_loss.item(), 'step',i,' | ',end='\r')
            count += 1
        return running_loss/count
    
    def val_epoch():
        print('no validation nerd')
    
    best_val_loss = 100000
    steps_since_improvement = 0
    hist = []
    best_weights = model.state_dict()
    print('model being saved to',save_path)
    for epoch in range(epochs):
        print('epoch',epoch)
        model.train(False)
        val_accs = eval_knn_model(model,validation_loader,klist=k_values,device=device)
        for kv, val_acc in zip(k_values,val_accs):
            print('val accuracy k=',kv,val_acc)
            
        model.train(True)
        avg_loss = train_epoch()
        print('train loss', avg_loss)
        #don't save immediately in case I cancel training
        if best_val_loss > avg_loss and epoch > 1:
            torch.save(model,save_path)
            best_weights = model.state_dict()
            best_val_loss = avg_loss
            steps_since_improvement = 0
        else:
            steps_since_improvement += 1
        
        hist_entry = {
            'epoch': epoch,
            'train_loss': avg_loss,
#             'train_acc':avg_acc,
#             'val_loss':avg_loss,
            'val_acc': val_accs,
            'lr': lr,
#             'loss_weights': '_'.join([str(l) for l in loss_weights])
        }
        hist.append(hist_entry)
        save_train_history(model,hist,root=root)
        if steps_since_improvement > patience:
            break
    return model,hist

m,h = train_model(
    UnsupervisedTripletEncoder(),
    all_labels,
    Constants.data_root,
    batch_size=50,
    histogram=False,
    lr=.0001,
)
del m
h

model being saved to ../../data/models/abstractmodel_balanced
epoch 0


/opt/conda/envs/rapids/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


val accuracy k= 3 [0.2579796016216278, 0.5969069004058838, 0.8458374738693237]
val accuracy k= 5 [0.2785455882549286, 0.6266864538192749, 0.8558736443519592]
val accuracy k= 7 [0.2867719829082489, 0.6335965991020203, 0.8550510406494141]
train loss 0.16926934211340158 step 876  |   
saved history to ../../data/results/history_abstractmodel.csv
epoch 1
val accuracy k= 3 [0.23132610321044922, 0.5435999035835266, 0.7333004474639893]
val accuracy k= 5 [0.2420204132795334, 0.5681145191192627, 0.7385653257369995]
val accuracy k= 7 [0.24152682721614838, 0.577163577079773, 0.7382363080978394]
train loss 0.0651183724499593step 877  |  |  
saved history to ../../data/results/history_abstractmodel.csv
epoch 2
val accuracy k= 3 [0.2181638777256012, 0.5435999035835266, 0.7165186405181885]
val accuracy k= 5 [0.23330043256282806, 0.5648239850997925, 0.7207963466644287]
val accuracy k= 7 [0.24119776487350464, 0.576340913772583, 0.7219480276107788]
